# 第10章：RAG进阶（Agentic RAG）

## 学习目标

本章将学习：
1. Agentic RAG vs 2-Step RAG的区别
2. Agent动态决策何时检索
3. 使用@tool创建retriever工具
4. RAG Agent的构建（create_agent）
5. 多数据源RAG（多个知识库）
6. 查询改写和优化技术
7. Hybrid RAG模式简介

## RAG的三种架构

### 1. 2-Step RAG（两步RAG）
- 固定流程：总是先检索，再生成
- 控制性强，延迟可预测
- 适合：FAQ、文档问答

### 2. Agentic RAG（智能体RAG）
- Agent决定何时检索
- 灵活性高，可多次检索
- 适合：复杂推理、多工具场景

### 3. Hybrid RAG（混合RAG）
- 结合两者优势
- 包含查询优化、结果验证等中间步骤
- 适合：需要质量控制的场景


In [2]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain.tools import tool
from langchain.agents import create_agent

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

# 配置Embedding模型
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL
)

print("环境配置完成！")
print(f"Chat模型: {model.model_name}")
print(f"Embedding模型: {embeddings.model}")

环境配置完成！
Chat模型: gpt-4.1-mini
Embedding模型: text-embedding-3-small


In [3]:
### 1.1 准备知识库

print("【准备知识库】")
print()

# 创建知识库文档
documents = [
    Document(page_content="LangChain是一个用于构建LLM应用的框架，提供了丰富的工具和组件。", metadata={"source": "langchain_intro"}),
    Document(page_content="Agent可以自主决策使用哪些工具，适合复杂的多步骤任务。", metadata={"source": "agent_guide"}),
    Document(page_content="RAG技术通过检索外部知识来增强LLM的回答能力。", metadata={"source": "rag_guide"}),
    Document(page_content="LCEL使用管道操作符|连接组件，构建复杂的处理链。", metadata={"source": "lcel_guide"}),
    Document(page_content="Tool Calling让LLM能够调用外部工具和API。", metadata={"source": "tools_guide"}),
]

# 创建向量存储
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(documents)

print(f"知识库已创建，包含 {len(documents)} 个文档")

【准备知识库】

知识库已创建，包含 5 个文档


## 2. Agentic RAG vs 2-Step RAG

### 核心区别

| 特性 | 2-Step RAG | Agentic RAG |
|------|-----------|-------------|
| 检索时机 | 总是检索 | Agent决定 |
| LLM调用次数 | 1次 | 多次（可变） |
| 控制性 | 高 | 低 |
| 灵活性 | 低 | 高 |
| 延迟 | 可预测 | 可变 |
| 适用场景 | 简单问答 | 复杂推理 |

### Agentic RAG的优势

- 只在需要时检索（节省成本）
- 可以多次检索（深度探索）
- 能处理问候、闲聊等不需要检索的场景
- Agent可以优化查询词

### Agentic RAG的劣势

- 需要多次LLM调用（成本高）
- 可能跳过必要的检索
- 延迟不可预测

## 3. 创建Retriever工具

将Retriever包装成Tool，让Agent可以调用。

In [12]:
print("【创建Retriever工具】")
print()

# 方式1：使用@tool装饰器包装retriever
@tool(response_format="content_and_artifact")
def retrieve_knowledge(query: str):
    """从知识库中检索相关信息。用于回答关于LangChain、RAG、Agent等技术问题。"""
    docs = vector_store.similarity_search(query, k=2)
    # 格式化为字符串
    content = "\n\n".join(
        f"来源: {doc.metadata['source']}\n内容: {doc.page_content}"
        for doc in docs
    )
    return content, docs  # content给LLM，docs作为artifact

# 测试工具
print("工具名称:", retrieve_knowledge.name)
print("工具描述:", retrieve_knowledge.description)
print()

# 直接调用测试
result = retrieve_knowledge.invoke({"query": "什么是RAG？"})
print("检索结果:")
print(result)

【创建Retriever工具】

工具名称: retrieve_knowledge
工具描述: 从知识库中检索相关信息。用于回答关于LangChain、RAG、Agent等技术问题。

检索结果:
来源: rag_guide
内容: RAG技术通过检索外部知识来增强LLM的回答能力。

来源: langchain_intro
内容: LangChain是一个用于构建LLM应用的框架，提供了丰富的工具和组件。



## 4. 构建Agentic RAG

使用`create_agent`创建能够自主决策何时检索的Agent。

In [13]:
print("【构建Agentic RAG】")
print()

# 创建Agent
tools = [retrieve_knowledge]
system_prompt = """你是一个AI助手，可以使用工具来检索知识库。

使用指南：
- 如果用户问候或闲聊，直接回答，不需要检索
- 如果问题需要知识库信息，使用retrieve_knowledge工具
- 如果第一次检索不够，可以再次检索
- 基于检索结果回答问题，并引用来源
"""

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt
)

print("Agentic RAG已创建！")
print(f"可用工具: {[t.name for t in tools]}")

【构建Agentic RAG】

Agentic RAG已创建！
可用工具: ['retrieve_knowledge']


### 4.1 测试场景1：不需要检索的问题

In [14]:
print("【测试：问候】")
print()

# Agent应该直接回答，不调用工具
response = agent.invoke({"messages": [{"role": "user", "content": "你好！"}]})

print("用户: 你好！")
print(f"Agent: {response['messages'][-1].content}")
print()

# 检查是否调用了工具
tool_calls = [msg for msg in response['messages'] if hasattr(msg, 'tool_calls') and msg.tool_calls]
print(f"是否调用工具: {'是' if tool_calls else '否'}")

【测试：问候】

用户: 你好！
Agent: 你好！有什么我可以帮您的吗？

是否调用工具: 否


### 4.2 测试场景2：需要检索的问题

In [15]:
print("【测试：需要检索的问题】")
print()

# Agent应该调用retrieve_knowledge工具
response = agent.invoke({"messages": [{"role": "user", "content": "什么是RAG技术？"}]})

print("用户: 什么是RAG技术？")
print()

# 显示完整的交互过程
for i, msg in enumerate(response['messages']):
    if hasattr(msg, 'content') and msg.content:
        role = msg.__class__.__name__
        print(f"[{role}]: {msg.content[:200]}...")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"[Tool Call]: {msg.tool_calls[0]['name']}(query='{msg.tool_calls[0]['args']['query']}')")

print()
print(f"最终回答: {response['messages'][-1].content}")

【测试：需要检索的问题】

用户: 什么是RAG技术？

[HumanMessage]: 什么是RAG技术？...
[Tool Call]: retrieve_knowledge(query='什么是RAG技术')
[ToolMessage]: 来源: rag_guide
内容: RAG技术通过检索外部知识来增强LLM的回答能力。

来源: agent_guide
内容: Agent可以自主决策使用哪些工具，适合复杂的多步骤任务。...
[AIMessage]: RAG技术（Retrieval-Augmented Generation）是一种通过检索外部知识来增强大型语言模型（LLM）回答能力的技术。它结合了检索和生成的能力，使模型能够利用外部知识库中的信息，提供更准确和丰富的回答。...

最终回答: RAG技术（Retrieval-Augmented Generation）是一种通过检索外部知识来增强大型语言模型（LLM）回答能力的技术。它结合了检索和生成的能力，使模型能够利用外部知识库中的信息，提供更准确和丰富的回答。


### 4.3 测试场景3：需要多次检索的复杂问题

In [18]:
print("【测试：复杂问题（可能多次检索）】")
print()

# Agent可能会多次调用工具
query = "Agent和LCEL分别是什么？它们有什么关系？"
response = agent.invoke({"messages": [{"role": "user", "content": query}]})

print(f"用户: {query}")
print()

# 统计工具调用次数
tool_call_count = sum(1 for msg in response['messages'] if hasattr(msg, 'tool_calls') and msg.tool_calls)
print(f"工具调用次数: {tool_call_count}")
print()
print(f"最终回答: {response['messages'][-1].content}")

【测试：复杂问题（可能多次检索）】

用户: Agent和LCEL分别是什么？它们有什么关系？

工具调用次数: 1

最终回答: Agent是LangChain框架中的一个概念，指的是能够根据用户输入自动选择和调用不同工具或组件来完成任务的智能体。它可以理解用户意图，动态地决定调用哪些工具，从而实现复杂的任务处理。

LCEL（LangChain Execution Language）是LangChain中用于构建复杂处理链的语言或机制。它通过使用管道操作符和连接组件，帮助开发者构建复杂的处理流程链条，实现多步骤的数据处理和任务执行。

它们的关系是：Agent作为智能体，可能会利用LCEL构建的复杂处理链来完成任务。LCEL提供了构建复杂处理流程的能力，而Agent则是利用这些流程和工具来智能地响应用户请求的执行者。

总结：
- Agent是智能体，负责理解和执行任务，调用工具。
- LCEL是构建复杂处理链的语言或机制。
- Agent可能会使用LCEL构建的处理链来完成任务。

以上信息来源于LangChain相关知识库。


## 5. 多数据源RAG

为不同的知识库创建不同的检索工具。

In [19]:
print("【多数据源RAG】")
print()

# 创建第二个知识库（Python编程）
python_docs = [
    Document(page_content="Python是一种高级编程语言，以简洁和可读性著称。", metadata={"source": "python_intro"}),
    Document(page_content="列表推导式是Python的特色语法，用于快速创建列表。", metadata={"source": "python_features"}),
    Document(page_content="装饰器是Python的高级特性，用于修改函数行为。", metadata={"source": "python_advanced"}),
]

python_vector_store = InMemoryVectorStore(embeddings)
python_vector_store.add_documents(python_docs)

# 为Python知识库创建工具
@tool(response_format="content_and_artifact")
def retrieve_python_docs(query: str):
    """从Python编程知识库中检索信息。用于回答Python语言相关问题。"""
    docs = python_vector_store.similarity_search(query, k=2)
    content = "\n\n".join(
        f"来源: {doc.metadata['source']}\n内容: {doc.page_content}"
        for doc in docs
    )
    return content, docs

# 创建多工具Agent
multi_tools = [retrieve_knowledge, retrieve_python_docs]
multi_system_prompt = """你是一个AI助手，可以访问两个知识库：
1. LangChain知识库（retrieve_knowledge）：关于LangChain、RAG、Agent等
2. Python知识库（retrieve_python_docs）：关于Python编程

根据问题选择合适的工具，或同时使用多个工具。
"""

multi_agent = create_agent(
    model=model,
    tools=multi_tools,
    system_prompt=multi_system_prompt
)

print(f"多数据源Agent已创建！")
print(f"可用工具: {[t.name for t in multi_tools]}")

【多数据源RAG】

多数据源Agent已创建！
可用工具: ['retrieve_knowledge', 'retrieve_python_docs']


### 5.1 测试多数据源检索

In [20]:
print("【测试：跨知识库问题】")
print()

# Agent需要选择正确的工具
query = "Python的装饰器是什么？"
response = multi_agent.invoke({"messages": [{"role": "user", "content": query}]})

print(f"用户: {query}")
print()

# 显示使用了哪个工具
for msg in response['messages']:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"调用工具: {tc['name']}")

print()
print(f"回答: {response['messages'][-1].content}")

【测试：跨知识库问题】

用户: Python的装饰器是什么？

调用工具: retrieve_python_docs

回答: Python的装饰器是一种高级特性，用于修改函数的行为。装饰器本质上是一个函数，它接受另一个函数作为参数，并返回一个新的函数，从而在不改变原函数代码的情况下，动态地增加或修改函数的功能。装饰器常用于日志记录、权限校验、性能测试等场景。你需要我详细讲解装饰器的用法和示例吗？


## 6. 查询改写与优化（Hybrid RAG）

Hybrid RAG在检索前对查询进行优化，提高检索质量。

In [21]:
print("【查询改写】")
print()

from pydantic import BaseModel

# 定义查询改写的输出结构
class RewrittenQuery(BaseModel):
    query: str

# 创建查询改写函数
def rewrite_query(original_query: str) -> str:
    """将用户查询改写为更适合检索的形式"""
    system_prompt = """你是查询优化专家。将用户的问题改写为更适合向量检索的形式。
    
改写原则：
- 提取关键概念和术语
- 去除口语化表达
- 保留核心语义
- 简洁明确

只返回改写后的查询，不要解释。"""
    
    response = model.with_structured_output(RewrittenQuery).invoke([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": original_query}
    ])
    return response.query

# 测试查询改写
original = "能不能告诉我RAG到底是啥东西？"
rewritten = rewrite_query(original)

print(f"原始查询: {original}")
print(f"改写查询: {rewritten}")
print()

# 对比检索效果
print("使用原始查询检索:")
docs1 = vector_store.similarity_search(original, k=2)
for doc in docs1:
    print(f"  - {doc.metadata['source']}: {doc.page_content[:50]}...")

print()
print("使用改写查询检索:")
docs2 = vector_store.similarity_search(rewritten, k=2)
for doc in docs2:
    print(f"  - {doc.metadata['source']}: {doc.page_content[:50]}...")

【查询改写】

原始查询: 能不能告诉我RAG到底是啥东西？
改写查询: RAG技术定义及应用

使用原始查询检索:
  - rag_guide: RAG技术通过检索外部知识来增强LLM的回答能力。...
  - langchain_intro: LangChain是一个用于构建LLM应用的框架，提供了丰富的工具和组件。...

使用改写查询检索:
  - rag_guide: RAG技术通过检索外部知识来增强LLM的回答能力。...
  - tools_guide: Tool Calling让LLM能够调用外部工具和API。...


## 7. 实战项目：智能RAG系统

构建一个完整的Hybrid RAG系统，结合查询改写和Agentic检索。

In [22]:
print("【智能RAG系统】")
print()

class SmartRAGSystem:
    """结合查询优化和Agentic检索的智能RAG系统"""
    
    def __init__(self, model, vector_store, embeddings):
        self.model = model
        self.vector_store = vector_store
        self.embeddings = embeddings
        self.setup_tools()
        self.setup_agent()
    
    def rewrite_query(self, query: str) -> str:
        """查询改写"""
        system_prompt = """改写查询使其更适合检索。提取关键词，去除口语化。"""
        response = self.model.with_structured_output(RewrittenQuery).invoke([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}
        ])
        return response.query
    
    def setup_tools(self):
        """创建检索工具（带查询优化）"""
        @tool(response_format="content_and_artifact")
        def smart_retrieve(query: str):
            """智能检索工具：自动优化查询并检索相关信息"""
            # 查询改写
            optimized_query = self.rewrite_query(query)
            
            # 检索
            docs = self.vector_store.similarity_search(optimized_query, k=3)
            
            # 格式化
            content = f"[查询优化: {query} -> {optimized_query}]\n\n"
            content += "\n\n".join(
                f"来源: {doc.metadata['source']}\n{doc.page_content}"
                for doc in docs
            )
            return content, docs
        
        self.tools = [smart_retrieve]
    
    def setup_agent(self):
        """创建Agent"""
        system_prompt = """你是智能问答助手。

使用smart_retrieve工具检索知识库（工具会自动优化查询）。
基于检索结果回答问题，并说明信息来源。
如果是问候或闲聊，直接回答。"""
        
        self.agent = create_agent(
            model=self.model,
            tools=self.tools,
            system_prompt=system_prompt
        )
    
    def query(self, question: str):
        """处理用户查询"""
        response = self.agent.invoke({"messages": [{"role": "user", "content": question}]})
        return response['messages'][-1].content

# 创建系统
rag_system = SmartRAGSystem(model, vector_store, embeddings)
print("智能RAG系统已创建！")

【智能RAG系统】

智能RAG系统已创建！


### 7.1 测试智能RAG系统

In [23]:
print("【测试智能RAG系统】")
print()

# 测试1：口语化问题
test_queries = [
    "能不能跟我说说RAG是啥？",
    "Agent和普通的LLM有啥区别？",
    "你好！"
]

for query in test_queries:
    print(f"问题: {query}")
    answer = rag_system.query(query)
    print(f"回答: {answer}")
    print("-" * 60)
    print()

【测试智能RAG系统】

问题: 能不能跟我说说RAG是啥？
回答: RAG（Retrieval-Augmented Generation）是一种技术，通过检索外部知识库中的信息来增强大型语言模型（LLM）的回答能力。简单来说，RAG结合了检索（Retrieval）和生成（Generation）两部分：先从外部知识库中检索相关信息，再基于这些信息生成更准确和丰富的回答。这种方法可以让模型在面对需要具体知识的问题时，提供更可靠和详细的答案。

信息来源于rag_guide中的相关介绍。
------------------------------------------------------------

问题: Agent和普通的LLM有啥区别？
回答: Agent和普通的LLM（大型语言模型）主要区别在于Agent具备调用外部工具和执行多步骤任务的能力，而普通的LLM主要是基于输入文本生成响应。

具体来说：
- 普通的LLM主要通过理解和生成文本来回答问题或完成任务，能力局限于模型本身的训练数据和推理能力。
- Agent则是在LLM基础上，结合了工具调用（Tool Calling）和多步骤推理的能力，可以主动调用外部API、数据库、检索系统等工具，完成更复杂的任务。例如，Agent可以先检索信息，再基于检索结果生成回答，或者调用计算工具进行计算。

这种设计使得Agent能够处理更复杂和动态的任务，扩展了LLM的应用场景。

信息来源包括LangChain框架介绍、RAG技术和工具调用相关资料。
------------------------------------------------------------

问题: 你好！
回答: 你好！有什么我可以帮您的吗？
------------------------------------------------------------



## 8. RAG架构对比总结

### 三种架构的选择指南

| 场景 | 推荐架构 | 原因 |
|------|---------|------|
| FAQ系统 | 2-Step RAG | 每个问题都需要检索，延迟可预测 |
| 文档问答 | 2-Step RAG | 简单高效，成本低 |
| 智能客服 | Agentic RAG | 需要处理闲聊和问答 |
| 复杂研究助手 | Agentic RAG | 需要多次检索和推理 |
| 多数据源查询 | Agentic RAG | Agent选择合适的数据源 |
| 需要高质量 | Hybrid RAG | 查询优化提高检索质量 |
| 对话式应用 | Agentic RAG | 灵活处理上下文 |

### 性能对比

**2-Step RAG:**
- LLM调用：1次
- 延迟：低（约1-2秒）
- 成本：低
- 控制性：高

**Agentic RAG:**
- LLM调用：2-5次（可变）
- 延迟：中高（约3-10秒）
- 成本：中高
- 灵活性：高

**Hybrid RAG:**
- LLM调用：2-3次
- 延迟：中（约2-5秒）
- 成本：中
- 质量：最高

### 实施建议

1. 从2-Step RAG开始（简单场景）
2. 如果需要灵活性，升级到Agentic RAG
3. 如果检索质量不够，添加查询优化（Hybrid）
4. 监控成本和延迟，按需调整

## 9. 总结与最佳实践

### 核心概念回顾

1. **2-Step RAG**: 固定的检索-生成流程，简单高效
2. **Agentic RAG**: Agent动态决策何时检索，灵活但成本高
3. **Hybrid RAG**: 结合查询优化和验证步骤，质量最高
4. **多数据源RAG**: 为不同知识库创建不同的工具
5. **查询改写**: 优化用户查询，提高检索质量

### Agentic RAG的关键要素

1. **Retriever工具**: 使用`@tool`装饰器包装检索器
2. **response_format**: 设置为`"content_and_artifact"`保留原始文档
3. **create_agent**: 创建能够使用工具的Agent
4. **System Prompt**: 指导Agent何时使用工具
5. **工具描述**: 清晰的描述帮助Agent选择正确工具

### 最佳实践

**工具设计：**
- 工具名称要清晰（如`retrieve_knowledge`）
- 描述要详细说明工具用途和适用场景
- 使用`response_format="content_and_artifact"`保留元数据
- 格式化输出，包含来源信息

**System Prompt：**
- 明确告诉Agent何时使用工具
- 说明不同工具的适用场景
- 指导Agent如何处理检索结果
- 提供示例和指导原则

**查询优化：**
- 提取关键词和术语
- 去除口语化和冗余表达
- 保持核心语义
- 使用结构化输出确保格式

**多数据源：**
- 为每个数据源创建独立工具
- 工具描述要明确数据源内容
- 让Agent自主选择合适的工具
- 可以并行调用多个工具

### 性能优化

1. **控制检索次数**: 在System Prompt中限制最大检索次数
2. **缓存常见查询**: 缓存热门问题的检索结果
3. **批量处理**: 对多个问题使用batch方法
4. **监控成本**: 跟踪LLM调用次数和token使用
5. **降级策略**: 高负载时切换到2-Step RAG

### 常见问题与解决

**问题1: Agent不调用工具**
- 检查System Prompt是否清晰
- 确认工具描述是否准确
- 测试不同的问题表述

**问题2: Agent过度调用工具**
- 在System Prompt中添加限制
- 优化工具描述，避免过于宽泛
- 调整temperature参数

**问题3: 检索质量不高**
- 添加查询改写步骤
- 调整检索参数（k值、相似度阈值）
- 改进文档切分策略
- 使用更好的Embedding模型

**问题4: 延迟太高**
- 考虑切换到2-Step RAG
- 减少检索文档数量
- 使用更快的模型
- 添加缓存层

**问题5: 成本太高**
- 监控并限制工具调用次数
- 使用更便宜的模型
- 添加缓存减少重复调用
- 对简单问题使用2-Step RAG

### 架构选择决策树

```
问题需要检索吗？
├─ 总是需要 → 2-Step RAG
└─ 不一定需要
   ├─ 只有一个数据源 → Agentic RAG
   └─ 多个数据源 → Agentic RAG + 多工具
   
检索质量是否足够？
├─ 是 → 保持当前架构
└─ 否 → 添加查询优化（Hybrid RAG）

延迟是否可接受？
├─ 是 → 保持当前架构
└─ 否 → 降级到2-Step RAG或优化
```

### 进阶方向

1. **Self-Query**: 从自然语言中提取结构化查询条件
2. **答案验证**: 检查生成答案的准确性和完整性
3. **多轮对话**: 支持上下文相关的连续提问
4. **混合检索**: 结合关键词和向量检索
5. **评估指标**: 使用RAGAS等工具评估RAG质量